# convert the trajtory of sz GPS data to trips format for fitting the kde algorithm

In [2]:
import pickle

with open('datasets/bus_long50_traj_wgs84_coor/0_24_gps.pkl', 'rb') as f:
    data = pickle.load(f)

data

,traj,id,time,lon,lat,angle,speed,device,dspeed,mileage
0,0,BS01983D,2019-10-09 07:03:56+00:00,113.911102,22.658583,256.0,15.0,1638196,0.0,48147.0
1,0,BS01983D,2019-10-09 07:04:10+00:00,113.910248,22.658617,279.0,12.0,1638196,0.0,48147.0
2,0,BS01983D,2019-10-09 07:04:14+00:00,113.910149,22.658617,274.0,0.0,1638196,0.0,48147.0
3,0,BS01983D,2019-10-09 07:04:28+00:00,113.909515,22.658617,271.0,28.0,1638196,0.0,48147.0
4,0,BS01983D,2019-10-09 07:04:29+00:00,113.909431,22.658634,272.0,27.0,1638196,0.0,48147.0
...,...,...,...,...,...,...,...,...,...,...
83259,2003,BS43022D,2019-10-09 21:47:40+00:00,113.911453,22.657784,159.0,14.0,1701105,0.0,53401.0
83260,2003,BS43022D,2019-10-09 21:47:45+00:00,113.911499,22.657700,155.0,0.0,1701105,0.0,53401.0
83261,2003,BS43022D,2019-10-09 21:47:59+00:00,113.911697,22.657484,141.0,16.0,1701105,0.0,53401.0
83262,2003,BS43022D,2019-10-09 21:48:02+00:00,113.911835,22.657316,143.0,20.0,1701105,0.0,53401.0


In [6]:
#convert the data time to timestamp

import pandas as pd
import os

# Load the pickle file
df = pd.read_pickle('datasets/bus_long50_traj_wgs84_coor/0_24_gps.pkl')

# Define the columns to keep
columns_to_keep = ['id', 'lat', 'lon', 'time']

# Get unique traj values
unique_trajs = df['traj'].unique()

# Save each unique traj to a separate text file
save_path = 'datasets/trips_0_24'
if not os.path.exists(save_path):
    os.makedirs(save_path)
for traj in unique_trajs:
    traj_df = df[df['traj'] == traj][columns_to_keep]
    traj_df['time'] = pd.to_datetime(traj_df['time']).apply(lambda x: x.timestamp())
    traj_df.to_csv(f'{save_path}/trip_{traj}.txt', header=False, index=False)
print('Done!')


Done!


In [4]:
import cv2
import numpy as np

def _direct_render(patchedGPS):
    """direct render GPS data to image"""
    gps = np.zeros((length, length, 1), np.uint8)
    ratio = length / 1024.
    patchedGPS = patchedGPS[(0 <= patchedGPS['2']) & (patchedGPS['2'] < 1024) &
                            (0 <= patchedGPS['1']) & (patchedGPS['1'] < 1024)]
    y = np.array(patchedGPS['2'] * ratio, np.int_)
    x = np.array(patchedGPS['1'] * ratio, np.int_)

    gps[x, y] = 255

    gps = cv2.dilate(gps, np.ones((3, 3))) # 膨胀
    gps = gps[..., None] # 增加一个维度
    return gps


In [5]:
import os
from PIL import Image
import pandas as pd

file_path = 'datasets/trips_0_24'
save_path = 'datasets/trips_0_24_image'
if not os.path.isdir(save_path):
    os.makedirs(save_path)
file_list = os.listdir(file_path)
for file_name in file_list:
    print(file_name)
    data = pd.read_csv(os.path.join(file_path, file_name), header=None)
    gps_image_array = _direct_render(data)
    cv2.imwrite(os.path.join(save_path, f"{file_name}.png"), gps_image_array)
    # gps_image = Image.fromarray(gps_image_array.squeeze())
    # gps_image.show()

print('Done!')

trip_322.txt


NameError: name 'length' is not defined